In [82]:
import pandas as pd
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import glob
from tqdm.notebook import tqdm
import os
from sklearn.metrics import f1_score

In [106]:
path = '../ignored/tvsum50_ver_1_1/ydata-tvsum50-v1_1/data/ydata-tvsum50-anno.tsv'
df = pd.read_csv(path,delimiter='\t', header=None, names=['id','cat','scores'])
df = df.join(df['scores'].str.split(',',expand=True).apply(pd.to_numeric))
df = df.drop(axis=1, columns=['scores'])

In [107]:
print(df[df['id']==file_id]['cat'])

580    PR
581    PR
582    PR
583    PR
584    PR
585    PR
586    PR
587    PR
588    PR
589    PR
590    PR
591    PR
592    PR
593    PR
594    PR
595    PR
596    PR
597    PR
598    PR
599    PR
Name: cat, dtype: object


In [110]:
# files = ['Bhxk-O1Y7Ho']
# files = ['98MoyGZKHXc', 'Bhxk-O1Y7Ho', '4wU_LUjG5Ic', 'iVt07TCkFM0']
files = []
for file_id in glob.glob('../inputs/*'):
    filename = file_id.split('/')[-1].split('.')[0]
    if filename not in files:
        files.append(filename)
proportion = 0.2

Get the predicted indices

In [111]:
temp = []
for file_id in files:
    frame_scores_for_video = df[df['id']==file_id].drop(axis=1, columns=['id','cat']).values
    frame_scores_for_video_nonan = frame_scores_for_video[~np.isnan(frame_scores_for_video)]
    frame_scores_for_video_nonan = frame_scores_for_video_nonan.reshape((20,-1))
    
    with open(f'../intermediate/{file_id}_clip_list.json') as f:
        clip_data = json.load(f)

    # Convert clip start time and end time to fps
    fps = round(cv2.VideoCapture(f'../inputs/{file_id}.mp4').get(cv2.CAP_PROP_FPS))
    for clip in clip_data:
        h,m,s = clip['clip_start'].split(':')
        secs = int(h)*3600+int(m)*60+float(s)
        clip['clip_start_fps'] = round(secs*fps)
        h,m,s = clip['clip_end'].split(':')
        secs = int(h)*3600+int(m)*60+float(s)
        clip['clip_end_fps'] = round(secs*fps)
        temp.append([clip['clip_start_fps'], clip['clip_end_fps']])
    # Generate list with included and excluded frames. Included frames are 1.
    val_list = [0 for i in range(frame_scores_for_video_nonan.shape[1])]
    try:
        for clip in clip_data:
            for i in range(clip['clip_start_fps'], clip['clip_end_fps']+1):
                val_list[i] = 1
        val_np = np.array(val_list)
    except:
        pass
    avg_frame_scores = frame_scores_for_video_nonan.mean(axis=0)
    fps = round(cv2.VideoCapture(f'../inputs/{file_id}.mp4').get(cv2.CAP_PROP_FPS))
    last = 0
    l = []
    for i,el in enumerate(list(frame_scores_for_video_nonan.mean(axis=0))):
        if i % (fps*2) == 0:
            d = {}
            last = i
            d['start'] = i
            d['score'] = el
            l.append(d)

    if last<len(avg_frame_scores):
        l.append({'start': len(avg_frame_scores)-1, 'score': avg_frame_scores[-1]})
    arranged = sorted(l,key=lambda x: x['score'], reverse=True)
    selected_list = [0 for i in range(len(avg_frame_scores))]
    target_length = round((len(avg_frame_scores))*proportion)
    for selected in arranged:
        for i in range(selected['start'],selected['start']+2*fps):
            try:
                selected_list[i] = 1
            except IndexError:
                pass
        target_length-=2*fps
        if target_length<0:
            break
    print(file_id, df[df['id']==file_id]['cat'].iloc[0], f1_score(y_true=selected_list, y_pred=val_list))

vdmoEJ5YbrQ VU 0.4982104509663566
HT5vyqe0Xaw VU 0.3489601691928093
eQu1rNs0an0 BT 0.3583977512297962
WxtbjNsCQ8A BK 0.39633027522935776
37rzWOQsNIw MS 0.3005405405405405
AwmHb44_ouw VT 0.3494757863205192
0tmA_C6XwfM GA 0.47765267826680313
b626MiF1ew4 PK 0.327007299270073
EYqVtI9YWJA BT 0.32410018245148453
jcoYJXDG9sw DS 0.3520610239108112
Se3oxnaPsz0 BK 0.35699107522311946
qqR6AEXwxoQ BT 0.36527621195039456
Bhxk-O1Y7Ho GA 0.2988314922418747
J0nA4VgnoCo VT 0.3698480773424438
XzYM3PfTM4w VT 0.40106100795755967
E11zDS9XGzg DS 0.3527416619559073
3eYKfiOEJNs GA 0.40630975143403436
JgHubY5Vw3Y BT 0.4297774366845741
i3wAGJaaktw GA 0.26460859977949286
WG0MBPpPC6I MS 0.31703204047217537
Yi4Ij2NM7U4 MS 0.3522411741372471
EE-bNr36nyA BK 0.338436744560838
iVt07TCkFM0 BT 0.40472440944881893
98MoyGZKHXc VT 0.3604629102637071
akI8YFjEmUw VU 0.3087213789554927
xwqBXPGE9pQ VU 0.4290822407628128
Hl-__g2gn_A MS 0.34408602150537637
4wU_LUjG5Ic PR 0.17526243724326793
